# Data loading, storage, and file formats

When dealing with large data sources, you can read them in piece-wise.

To make the visualisation of such files handier, you can set the display options:

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [3]:
pd.options.display.max_rows = 10

## Reading in chunks

To read a file in chunks, specify the chunksize:

`chunker = pd.read_csv(example.csv, chunksize=1000)`

Afterwards, you can iterate over such chunks

In [13]:
chunker = pd.read_csv('wesm/examples/ex6.csv', chunksize=1000)
chunker

In [14]:
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot.sort_values(ascending=False, inplace=True)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## Writing data

You can write DataFrames and Series to CSV files using `to_csv`.

it's possible to write a subset of columns

`df.to_csv('file', index=False, columns=['n1, n2, ... n']`


## Working with delimited formats

Sometimes, you can't just parse files because they contain erros (ex7)

In [15]:
import csv

In [23]:
f = open('wesm/examples/ex7.csv')
reader = csv.reader(f)
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [24]:
with open('wesm/examples/ex7.csv') as f:
    lines = list(csv.reader(f))
header, values = lines[0], lines[1:]

data_dict = {h:v for h,v in zip(header, zip(*values))} #zip(*values) transposes rows - columns
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [25]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter=';'
    quotechar='"'
    quoting= csv.QUOTE_MINIMAL


In [28]:
f = open('wesm/examples/ex7.csv')
reader = csv.reader(f, dialect=my_dialect)


## Json data

* `pandas.read_json()`
* `df.to_json()`
* `series.to_json()`

## XML / HTML scraping

Pandas comes with some built-in tools for web scraping!

In [29]:
tables = pd.read_html('wesm/examples/fdic_failed_bank_list.html')
len(tables)

1

In [30]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


## HDF5

"Hierarchical Data Format" can be used for storing large amounts of (array / scientific) data. 
You can efficiently read / write small sections of much larger sets of data using it.

Pandas includes utility functions to access the data.

In [7]:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [8]:
store['obj1']

,a
0,-0.614214
1,-2.663250
2,-0.173278
3,0.364346
4,1.138053
...,...
95,1.773109
96,-1.736972
97,-0.753595
98,-0.140008


HDF supports two storage schemas, "fixed" and "table". The latter is generally slower, but it supports query operations using a special syntax:

In [10]:
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,0.336222
11,0.041614
12,0.359484
13,-1.709181
14,-0.012209
15,1.698826


** HDFS should not be used as a DB! Files can corrupt when many people write to it! **